<a href="https://colab.research.google.com/github/LydiaDEdou/projet/blob/main/spam_message_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sujet : spam message classification**




# **Importation des bibliotheque**

In [ ]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import nltk
from nltk.corpus import stopwords
import spacy
from sklearn.feature_selection import SelectKBest, chi2

In [ ]:
# Télécharger les stopwords Anglais
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# Charger le modèle spaCy pour la lemmatisation en français
nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Choix des étapes de prétraitement pour la classification d'email avec Naive Bayes**

Chaque étape de prétraitement a un rôle spécifique, et l'ordre est essentiel pour produire une représentation textuelle propre et cohérente. Voici les étapes, de leur ordre recommandé, et des explications de leur utilité dans la classification de spam/ham que j'ai utlisé.


*   **Lowercasing **: Convertissez simplement le en minuscules avec .lower()
*   **Gestion des entités spéciales (Emails, Numéros de téléphone)**
        **Pourquoi :** Les spams contiennent souvent des informations comme des numéros de téléphone ou des adresses email. Remplacer ces entités par des tokens génériques ([EMAIL], [PHONE]) permet au modèle de reconnaître ces éléments sans surcharger le vocabulaire.
        **Recommandé :** Oui, surtout pour des tâches de spam où ces entités peuvent indiquer un spam potentiel.



*  **Suppression de la ponctuation:** cela permet de simplifier la représentation textuelle.
*  **Tokenization:** Diviser le texte en unités individuelles appelées tokens (mots en général).


*  **Suppression des stopwords** Éliminer les mots courants (comme "le", "de", "et" en anglais) qui n’apportent pas de valeur informative dans la classification.
* **Lemmatisation**: Effectuez cette étape après les autres prétraitements pour travailler sur les mots nettoyés et filtrés.







In [ ]:
# Fonction de prétraitement avec gestion des entités spéciales
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

# Remplacement des entités spécifiques
    text = re.sub(r'\S+@\S+', ' [EMAIL] ', text)       # Remplace les emails
    text = re.sub(r'\b\d{10}\b', ' [PHONE] ', text)    # Remplace les numéros de téléphone
    text = re.sub(r'http\S+|www\S+', ' [URL] ', text)  # Remplace les URL
    text = re.sub(r'\$\d+|\d+\s?€', ' [MONEY] ', text) # Remplace les montants en $ ou €
    text = re.sub(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b', ' [DATE] ', text) # Remplace les dates
    text = re.sub(r'\b\d+\b', ' [NUMBER] ', text)      # Remplace les séquences de chiffres
    text = re.sub(r'\b\d{1,2}(:\d{2})?\s?(AM|PM|am|pm)?\b', ' [TIME] ', text) # Remplace les heures
    text = re.sub(r'@\w+', ' [MENTION] ', text)        # Remplace les mentions
    text = re.sub(r'#\w+', ' [HASHTAG] ', text)        # Remplace les hashtags
    text = re.sub(r'[&%$#@*!^]', ' [SYMBOL] ', text)   # Remplace les symboles spéciaux


    # Suppression de la ponctuation
    text = re.sub(r'[^\w\s]', ' ', text)

    # Tokenization et suppression des stopwords
    tokens = [word for word in text.split() if word not in stop_words]

    # Lemmatisation
    doc = nlp(" ".join(tokens))
    tokens = [token.lemma_ for token in doc]

    return " ".join(tokens)


# **Charger les données**

In [ ]:
# Charger le dataset
df = pd.read_csv("/content/MyBalancedSpamClassificationDataset.csv")
df['message'] = df['Message'].apply(preprocess_text)

In [ ]:

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)


#**Construire la pipeline NLP**

La vectorisation des textes avec TfidfVectorizer.
L'entraînement du modèle Multinomial Naive Bayes.

In [ ]:


# Ajout de n-grams dans le vectorizer
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),  # Utilisation de bigrams
    ('nb', MultinomialNB())
])





# **Évaluer le modèle**

In [ ]:
# Entraîner le modèle
pipeline.fit(X_train, y_train)

# Prédictions
y_pred = pipeline.predict(X_test)


# Calcul de la précision, du rappel et du F1 score
precision = precision_score(y_test, y_pred, pos_label="spam")
recall = recall_score(y_test, y_pred, pos_label="spam")
f1 = f1_score(y_test, y_pred, pos_label="spam")  # Calcul du F1 score

# Affichage des résultats
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(f"F1 score: {f1:.4f}")  # Affichage du F1 score
print(f"Rappel (recall): {recall:.4f}")  # Affichage du rappel




Classification Report:
              precision    recall  f1-score   support

         ham       0.99      0.96      0.98       422
        spam       0.97      0.99      0.98       475

    accuracy                           0.98       897
   macro avg       0.98      0.98      0.98       897
weighted avg       0.98      0.98      0.98       897

F1 score: 0.9802
Rappel (recall): 0.9916


In [ ]:


# Phrase à tester
sentence = "A major is sending you refunding of 30$, Please reply with your bank account information."

# Prétraitement de la phrase
processed_sentence = preprocess_text(sentence)



# Utilisation du modèle pour obtenir les probabilités
probabilities = pipeline.predict_proba([processed_sentence])[0]

# Affichage des résultats
spam_probability = probabilities[1]  # Probabilité de la classe "spam" (indice 1)
non_spam_probability = probabilities[0]  # Probabilité de la classe "non-spam" (indice 0)

print(f"Probabilité de SPAM : {spam_probability * 100:.2f}%")
print(f"Probabilité de NON SPAM : {non_spam_probability * 100:.2f}%")

# Utilisation du modèle pour prédire
prediction = pipeline.predict([processed_sentence])[0]

# Affichage du résultat
if prediction == "spam":
    print("La phrase est classée comme : SPAM")
else:
    print("La phrase est classée comme : NON SPAM")


Probabilité de SPAM : 89.00%
Probabilité de NON SPAM : 11.00%
La phrase est classée comme : SPAM
